In [ ]:

import keras
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras.backend as K
import matplotlib.pyplot as pyplot
import tensorflow as tf
from keras import models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical
from numpy.random import seed
from tensorflow import keras
keras.__version__
from keras import layers
from keras import regularizers
from csv import writer
from csv import reader
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
seeds = list(range(0,10))
multiply= [2,1,0.5]
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
data = pd.read_csv('.\Abalone_seed_result/abalone.csv')
print(data)

# One hot enconding
i=0; 
for sex in data['Sex'].unique():
            data['Sex']=data['Sex'].replace(to_replace=sex,value=i)
            i=i+1

X=data.iloc[:,0:8]
y=data.iloc[:,8].values

scalar= MinMaxScaler()
X= scalar.fit_transform(X)
y= y.reshape(-1,1)
y=scalar.fit_transform(y)

for mltp in multiply:
    for actual_seed in seeds:
        seed(actual_seed)
        tf.random.set_seed(actual_seed)


        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=actual_seed)

        model = Sequential()
        model.add(Dense(units=6*mltp, input_dim=8,activation='relu'))
        model.add(Dense(units=6*mltp, activation='relu'))
        model.add(Dense(units=6*mltp ,activation='relu'))
        model.add(Dense(units=1,activation='linear'))
        
        model.compile(optimizer="adam", 
                      loss='mean_squared_error')
        
        filename =".\Abalone_seed_result/"+ "Abalone_" + str(mltp) + "_" + str(actual_seed) + ".csv"
        history_logger= keras.callbacks.CSVLogger(filename, separator=",", append=False)
        time_callback = TimeHistory()
        history=model.fit(X_train,y_train,
                          batch_size=32, 
                          validation_data=(X_test,y_test),
                          epochs=100,
                          callbacks=[history_logger,time_callback])

        times = time_callback.times
        csv_input = pd.read_csv(filename)
        csv_input['time'] = times
        csv_input.to_csv(filename, index=False)
        K.clear_session()
        del model
        tf.compat.v1.reset_default_graph()